In [ ]:
# 크롤링 밴당함 위험 : 창섭쌥 : 아마존 자료를 캐글에서 본듯함 -> 뒤지기

In [13]:
# 우선 각 책 페이지의 내용은 동적이 아님. 개꿀!

# 각 페이지별 웹 크롤링
def book_info_page(url):
    import requests

    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }

    r = requests.get(url, headers=headers)

    test_book_html = r.text

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(test_book_html, 'html.parser')

    # ResultSet 객체를 가져온 후, 각 요소에 대해 select 메서드 호출
    book_info= soup.find('div', class_='bookCatalog_book_catalog__yiiIc')

    try:
        # 책 제목
        book_info_head=book_info.find('div', class_='bookCatalog_book_info_top__SUILS')
        book_summary=book_info_head.find('div', class_='bookSummary_book_summary__NsCmt')
        book_title_outline=book_summary.find('div', class_='bookTitle_book_title__e3mof')
        book_title_area=book_title_outline.find('div', class_='bookTitle_title_area__fspvB')
        title=book_title_area.find('h2', class_='bookTitle_book_name__JuBQ2') #제목

        title.text


        # 저자
        book_detail=book_summary.find('div', class_='bookTitle_detail_area__0yrpT')
        ul_book_detail=book_detail.find('ul', class_='bookTitle_list_info__yXUyF')
        writer_book_detail=ul_book_detail.find_all('li', class_='bookTitle_item_info__IMLyY')
        writer=writer_book_detail[0].find('div', class_='bookTitle_info_content__iHfCC')
        writer.text

        # 출판
        public_book_deatil=ul_book_detail.find_all('li', class_='bookTitle_item_info__IMLyY')
        public_company=public_book_deatil[2].find('div', class_='bookTitle_info_content__iHfCC')
        public_company.text

        # 발행
        public_book_deatil=ul_book_detail.find_all('li', class_='bookTitle_item_info__IMLyY')
        public_date=public_book_deatil[3].find('div', class_='bookTitle_info_content__iHfCC')
        public_date.text


        # 책리뷰와 갯수
        book_review=book_summary.find('div', class_='bookReview_book_review__lo_X6')
        book_review_atag=book_review.find('a', class_='bookReview_link_review__flLFj')
        book_review_num=book_review_atag.find('span', class_='bookReview_num__WC_Im')
        book_review_num.text #책 리뷰 갯수

        book_review_grade=book_review_atag.find('span', class_='bookReview_grade__0kV7o')
        book_review_grade.text #평점 점수

        # 책소개
        bookCatag=book_info.find('div', class_='bookCatalog_contents__Qr2wA')
        bookCatag_main_info=bookCatag.find('div', class_='bookMainInfo_main_info__SnQra')
        info_item=bookCatag_main_info.find_all('div', class_='infoItem_info_item__1Nm8P')
        info_item_data=info_item[0].find('div', class_='infoItem_data_box__VwBf3')
        info_item_text=info_item_data.find('div', class_='infoItem_data_text__bUgVI')
        info_item_text.text

        # 출판사 서평
        company_item_data=info_item[1].find('div', class_='infoItem_data_box__VwBf3')
        company_item_text=company_item_data.find('div', class_='infoItem_data_text__bUgVI')
        company_item_text.text

        # 책 카테고리
        book_basic_info=bookCatag_main_info.find('div', class_='bookBasicInfo_basic_info__HCWyr')
        book_basic_ul=book_basic_info.find('ul', class_='bookBasicInfo_list_info__2zETc')
        book_basic_li=book_basic_ul.find('li', class_='bookBasicInfo_item_info__Iebpg')
        boo_category=book_basic_li.find('div', class_='bookBasicInfo_info_detail__I0Fx5')
        boo_category.text

    except:
        return ""

    book_info_dic={
        '책 제목' : title.text,
        '저자' : writer.text,
        '출판' : public_company.text,
        '발행' : public_date.text,
        '책 리뷰 갯수' : book_review_num.text,
        '평점 점수' : book_review_grade.text,
        '책 소개' : info_item_text.text,
        '출판사 서평' : company_item_text.text,
        '책 카테고리' : boo_category.text
    }

    return book_info_dic
#
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless')
service = ChromeService(executable_path='C:/psm/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)


#url리스트 만드는 코드부분
url_list = []
sample_url = 'https://search.shopping.naver.com/book/search/category?bookTabType=ALL&catId=50005754&pageIndex=1&pageSize=40'

for change_num in range(11, 26):
    split1_list = sample_url.split('=')
    split2_list = split1_list[3].split('&')

    url = f'{split1_list[0]}={split1_list[1]}={split1_list[2]}={change_num}&{split2_list[1]}={split1_list[4]}'

    url_list.append(url)

root_dic={}
for url in url_list:

    target_url=url

    driver.implicitly_wait(30)
    driver.get(target_url)

    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)

    list_book=driver.find_element(By.CLASS_NAME,'list_book')
    list_book_li=list_book.find_elements(By.CLASS_NAME,'bookListItem_item_book__1yCey')

#     root_dic={}
    # list_book_li 순회
    for book in list_book_li:
        a_tag=book.find_element(By.TAG_NAME,'a')
        url=a_tag.get_attribute('href') #url

        bookList_item=a_tag.find_element(By.CLASS_NAME,'bookListItem_text_area__hF892')
        title=bookList_item.find_element(By.CLASS_NAME,'bookListItem_text__bglOw')
        title_text = title.text #책 이름

        result_data = book_info_page(url)

        root_dic[title_text]={'url': url, 'result': result_data}


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".list_book"}
  (Session info: headless chrome=118.0.5993.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF695578EF2+54786]
	(No symbol) [0x00007FF6954E5612]
	(No symbol) [0x00007FF69539A64B]
	(No symbol) [0x00007FF6953DB79C]
	(No symbol) [0x00007FF6953DB91C]
	(No symbol) [0x00007FF695416D87]
	(No symbol) [0x00007FF6953FBEAF]
	(No symbol) [0x00007FF695414D02]
	(No symbol) [0x00007FF6953FBC43]
	(No symbol) [0x00007FF6953D0941]
	(No symbol) [0x00007FF6953D1B84]
	GetHandleVerifier [0x00007FF6958C7F52+3524194]
	GetHandleVerifier [0x00007FF69591D800+3874576]
	GetHandleVerifier [0x00007FF695915D7F+3843215]
	GetHandleVerifier [0x00007FF695615086+694166]
	(No symbol) [0x00007FF6954F0A88]
	(No symbol) [0x00007FF6954ECA94]
	(No symbol) [0x00007FF

In [ ]:
import json
json_string = json.dumps(root_dic, ensure_ascii=False)

# JSON 데이터를 파일에 저장
output_file_name = '네이버 영미소설 book크롤링1.json'

with open(output_file_name, 'w', encoding='utf-8') as json_file:
    json.dump(root_dic, json_file, ensure_ascii=False)

In [11]:
print(root_dic)

{'어서 오세요, 휴남동 서점입니다 (황보름 장편소설)': {'url': 'https://cr3.shopping.naver.com/book/adcr?x=HftORYeyfG61IIyNJqTqXf%2F%2F%2Fw%3D%3DsfA51UMcEzFPRjjtMX0iMgNEL4%2FonNhUB1Wwl2yh4f3bAVynIKybXoqYA4swaWthyqBYLA3%2B8qYwSN6VeDdYCpd1ajMC%2B9PJWm6A3VzJOUawPHVI64G5UiqA8O5lzv4otRsCh5GS%2BklokEzA%2FJ0VszxCGSQdcrj%2BZJHLmJxgT%2FbsVNNGtEXa3gAsx%2Ff1Mm1u4dRiEl%2Bglva%2F91DgxtDeN0%2FyBgiVKIzr5%2BKs97R4s5Os%2Fa18qhDQ1A%2FKJp3fgBDorzDdPlS2xREE5SW0evKBllLl0AW9EIustvV8X%2Ba9MoHmoFqVqevD5hHRxMIf9lRgvDX094t%2FZzzd2ttckAVUmyRkUTykRCCDu6cTfSjTQnZsF2cWtqHLjE5VHhzheg8O%2BP0txwPH9KEAQLP9%2FUewQ9BWJX%2Bo3i%2FvsTM5MScvUvRMy3TxZ4GnjqSTypj1vXQ1ZFLhPKzCFzneRPRp9rFXPX5tGbfyJPJbQwi6UbDxGRVfP2iTor9sLYAfopiYPxQMozIVQSDpeCCW1qorKcJfeVXWJLFl6t4XkmZn177chzkU%3D&nvMid=32477571968&catId=50010002', 'result': ''}, '지구 끝의 온실 (여름 에디션,김초엽 장편소설)': {'url': 'https://cr3.shopping.naver.com/book/adcr?x=2Nn3hcRiI3GdztIJSGmzcP%2F%2F%2Fw%3D%3DsMEI%2BNcCt9BmmkJrhRCwL4dyRLe2F0IvZpOrxjJiAL5D%2BkDXDB5LhQnDnQjq6001yTk5vrXPXTBrPc2fq0VXHpN1ajMC%2B9PJWm6A3